<a href="https://colab.research.google.com/github/Gongoya/Big_Balls/blob/master/FinalProjectADS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pycaret[full]

     |████████████████████████████████| 301 kB 14.2 MB/s 
     |████████████████████████████████| 261 kB 60.9 MB/s 
     |████████████████████████████████| 167 kB 59.9 MB/s 
     |████████████████████████████████| 1.7 MB 60.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 6.8 MB 77.5 MB/s 
     |████████████████████████████████| 15.5 MB 61.8 MB/s 
     |████████████████████████████████| 56 kB 5.4 MB/s 
     |████████████████████████████████| 2.0 MB 71.3 MB/s 
     |████████████████████████████████| 114 kB 65.5 MB/s 
     |████████████████████████████████| 1.3 MB 49.1 MB/s 
     |████████████████████████████████| 54 kB 3.6 MB/s 
     |████████████████████████████████| 865 kB 57.9 MB/s 
     |████████████████████████████████| 100 kB 10.0 MB/s 
     |██████████████████

In [2]:
pip install fastapi

In [4]:
import pycaret

In [7]:
#load dataset from pyret
from pycaret.datasets import get_data
data =get_data('diamond')

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


EXPLORATORY DATA ANALYSIS

In [6]:
#plot carat_weight and price
import plotly.express as px
fig = px.scatter(x=data['Carat Weight'], y=data['Price'], 
                 facet_col = data['Cut'], opacity = 0.25, template = 'plotly_dark', trendline='ols',
                 trendline_color_override = 'red', title = 'SARAH GETS A DIAMOND - A CASE STUDY')
fig.show()

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [9]:
#Checking distribution of the target variable, Price
# plot histogram
fig = px.histogram(data, x=["Price"], template = 'plotly_dark', title = 'Histogram of Price')
fig.show()

In [10]:
#Notice that distribution of Price is right-skewed, we can quickly check to see if log transformation can make Price approximately normal to give fighting chance to algorithms that assume normality.
import numpy as np
# create a copy of data
data_copy = data.copy()
# create a new feature Log_Price
data_copy['Log_Price'] = np.log(data['Price'])
# plot histogram
fig = px.histogram(data_copy, x=["Log_Price"], title = 'Histgram of Log Price', template = 'plotly_dark')
fig.show()

DATA PREPARATION
PyCaret, the setup is the first and the only mandatory step in any machine learning experiment performed in PyCaret. This function takes care of all the data preparation required prior to training models.
PyCaret also offers a wide array of pre-processing features. To learn more about all the preprocessing functionalities in PyCaret, https://pycaret.org/preprocessing/

In [11]:
# init setup
from pycaret.regression import *

# Whenever you initialize the setup function in PyCaret, it profiles the dataset and infers the data types for all input features

s = setup(data, target = 'Price', transform_target = True)

#   transform_target = True:   PyCaret will transform the Price variable behind the scene using box-cox transformation

,Description,Value
0,session_id,6261
1,Target,Price
2,Original Data,"(6000, 8)"
3,Missing Values,False
4,Numeric Features,1
5,Categorical Features,6
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(4199, 28)"


MODEL TRAINING & SELECTION

In [12]:
# compare all models
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,557.8830,1.296946e+06,1112.3580,0.9876,0.0605,0.0444,1.418
xgboost,Extreme Gradient Boosting,634.8613,1.659703e+06,1280.1386,0.9837,0.0679,0.0502,4.746
lightgbm,Light Gradient Boosting Machine,621.0143,2.015885e+06,1376.7003,0.9810,0.0662,0.0480,0.143
rf,Random Forest Regressor,705.8187,2.229629e+06,1458.6267,0.9783,0.0771,0.0565,1.070
et,Extra Trees Regressor,720.8112,2.331653e+06,1494.6157,0.9774,0.0800,0.0586,1.173
gbr,Gradient Boosting Regressor,728.7075,2.439604e+06,1490.9372,0.9774,0.0756,0.0562,0.289
dt,Decision Tree Regressor,909.9339,3.379004e+06,1814.6693,0.9667,0.0999,0.0735,0.039
ada,AdaBoost Regressor,1974.6745,1.672152e+07,4044.7375,0.8389,0.1890,0.1509,0.248
knn,K Neighbors Regressor,2999.7416,3.551449e+07,5946.4444,0.6484,0.3589,0.2283,0.086
omp,Orthogonal Matching Pursuit,3333.6544,9.123849e+07,9297.1479,0.1067,0.2847,0.2232,0.024


In [13]:
# check the residuals of trained model
plot_model(best, plot = 'residuals_interactive')